In [1]:
# Importing our dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
# Reading into the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

# ATTEMPT #1: 
We will pre-process our data similarly to the model in Jupyter Notebook, and the change will be in the number of nodes for each of the 2 layers in our model below.

In [3]:
# Dropping the non-beneficial ID columns, 'EIN' and 'NAME'.
application1_df = application_df.drop(columns= ['EIN', 'NAME'])

In [4]:
# Determining the number of unique values in each column.
application1_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Looking at APPLICATION_TYPE value counts for binning
application1_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choosing 500 as a cutoff value and creating a list of application types to be replaced
application_types_to_replace = ['T9','T13','T12','T2','T25', 'T14', 'T29', 'T15', 'T17']

# Replacing the above by 'other' in the dataframe
for app in application_types_to_replace:
    application1_df['APPLICATION_TYPE'] = application1_df['APPLICATION_TYPE'].replace(app,"Other")

# Checking to make sure binning was successful
application1_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Looking at CLASSIFICATION value counts for binning
classification = application1_df['CLASSIFICATION'].value_counts()

In [8]:
# Converting the series to a dataframe in order to make a list of the classification we would like to replace
df1 = classification.to_frame(name='count').reset_index(names='class')

In [9]:
# Creating the list of classification we would like to replacing, choosing 1000 as a cutoff value
classifications_to_replace = []

for i in df1.index:
    if df1.loc[i]['count'] < 1000:
        classifications_to_replace.append(df1.loc[i]['class'])

In [10]:
# Replacing the above in the dataframe
for cls in classifications_to_replace:
    application1_df['CLASSIFICATION'] = application1_df['CLASSIFICATION'].replace(cls,"Other")
    
# Checking to make sure binning was successful
application1_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Converting categorical data to numeric with `pd.get_dummies`
enc_application1_df = pd.get_dummies(application1_df)

In [12]:
enc_application1_df.head()

STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  ...   
2                    0                    1                    0  ...   
3                    0                    0                    0  ...   
4                    0                    0                    0  ...   

   INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                  0                       0                         0   
1                  1                       0                         0   
2                  0                       0                         0   
3                  0                       1                         0   
4                  0                       0                         1   

   INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  \
0                   0                 0                       0   
1                   0                 0                       0   
2                   0                 0                       0   
3                   0                 0                       0   
4                   0                 0                       0   

   INCOME_AMT_50M+  INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  \
0                0                  0                         1   
1                0                  0                         1   
2                0                  0                         1   
3                0                  0                         1   
4                0                  0                         1   

   SPECIAL_CONSIDERATIONS_Y  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 44 columns]

In [13]:
# Splitting our preprocessed data into our features and target arrays
y = enc_application1_df['IS_SUCCESSFUL'].values
X = enc_application1_df.drop(['IS_SUCCESSFUL'],1).values 

# Splitting the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 1)

<ipython-input-13-4388c1a2bf4f>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = enc_application1_df.drop(['IS_SUCCESSFUL'],1).values


In [14]:
# Creating a StandardScaler instances
scaler = StandardScaler()

# Fitting the StandardScaler
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model



In [15]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 9,501
Trainable params: 9,501
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Training the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 6ms/step - loss: 0.5701 - accuracy: 0.7212
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5563 - accuracy: 0.7284
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7297
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7324
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5468 - accuracy: 0.7347
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accura

In [18]:
# Evaluating the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"1st attempt Loss: {model_loss}, 1st attempt Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5589 - accuracy: 0.7290 - 400ms/epoch - 1ms/step
1st attempt Loss: 0.5588715076446533, 1st attempt Accuracy: 0.7289795875549316




---



---



---



# ATTEMPT #2: 
Here again, data pre-processing will be kept the same, and the change will be increasing the number of layers and nodes in each layer in our model below.

In [19]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features_2 = len(X_train[0])
hidden_nodes_layer1_2 =  100
hidden_nodes_layer2_2 = 100
hidden_nodes_layer3_2 = 100
hidden_nodes_layer4_2 = 100
hidden_nodes_layer5_2 = 100

nn = tf.keras.models.Sequential()

# 1st hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer1_2, activation="relu", input_dim=number_input_features_2))

# 2nd hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="relu"))

# 3rd hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_2, activation="relu"))

# 4th hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4_2, activation="relu"))

# 5th hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5_2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               4400      
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 44,901
Trainable params: 44,901
Non-trai

In [20]:
# Compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Training the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5682 - accuracy: 0.7238
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5567 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5531 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5512 - accuracy: 0.7330
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5498 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5483 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5475 - accuracy: 0.7341
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accura

In [22]:
# Evaluating the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"2nd attempt Loss: {model_loss}, 2nd attempt Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5766 - accuracy: 0.7306 - 1s/epoch - 4ms/step
2nd attempt Loss: 0.5765700936317444, 2nd attempt Accuracy: 0.7306122183799744




---



---



---



# ATTEMPT #3: 
Here again, data pre-processing will be kept the same, and the change will be using different activation functions for the hidden layers in our model below.

In [23]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features_3 = len(X_train[0])
hidden_nodes_layer1_3 =  80
hidden_nodes_layer2_3 = 30


nn = tf.keras.models.Sequential()

# 1st hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer1_3, activation="tanh", input_dim=number_input_features_3))

# 2nd hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3520      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Training the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5718 - accuracy: 0.7214
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5571 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7282
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5507 - accuracy: 0.7306
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7325
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7330
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7343
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accura

In [26]:
# Evaluating the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"3rd attempt Loss: {model_loss}, 3rd attempt Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5575 - accuracy: 0.7303 - 397ms/epoch - 1ms/step
3rd attempt Loss: 0.557525098323822, 3rd attempt Accuracy: 0.7302623987197876


# ATTEMPT #4: 
Here again, data pre-processing will be kept the same, and the change will be decreasing the epochs in our model below.

In [27]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features_4 = len(X_train[0])
hidden_nodes_layer1_4 =  80
hidden_nodes_layer2_4 = 30


nn = tf.keras.models.Sequential()

# 1st hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer1_4, activation="relu", input_dim=number_input_features_4))

# 2nd hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                3520      
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Training the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=70)

Epoch 1/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5708 - accuracy: 0.7193
Epoch 2/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7281
Epoch 3/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7282
Epoch 4/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7283
Epoch 5/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7305
Epoch 6/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7324
Epoch 7/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7318
Epoch 8/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7302
Epoch 9/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7338
Epoch 10/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7318

In [30]:
# Evaluating the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"4th attempt Loss: {model_loss}, 4th attempt Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5536 - accuracy: 0.7278 - 537ms/epoch - 2ms/step
4th attempt Loss: 0.5535637140274048, 4th attempt Accuracy: 0.7278134226799011




---



---



---



# ATTEMPT #5: 
This time, we will change our data pre-processing by creating more bins for rare occurences in "APPLICATION_TYPE", "CLASSIFICATION" and "ASK_AMT" columns, while keeping the rest similar to the Jupyter Notebook model.

In [31]:
application5_df = application_df.copy()

In [32]:
# Determining the number of unique values in each column.
application5_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [33]:
# Dropping the non-beneficial ID columns, 'EIN' and 'NAME'.
application5_df = application5_df.drop(columns= ['EIN', 'NAME'])

In [34]:
# Choosing 10 as a cutoff value and creating a list of application types to be replaced
application_types_to_replace = ['T25', 'T14', 'T29', 'T15', 'T17']

# Replacing the above by 'other' in the dataframe
for app in application_types_to_replace:
    application5_df['APPLICATION_TYPE'] = application5_df['APPLICATION_TYPE'].replace(app,"Other")

# Checking to make sure binning was successful
application5_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [35]:
# Looking at CLASSIFICATION value counts for binning
classification = application5_df['CLASSIFICATION'].value_counts()
print(classification)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [36]:
# Converting the series to a dataframe in order to make a list of the classification we would like to replace
class_df = classification.to_frame(name='count').reset_index(names='class')
class_df.head()

class  count
0  C1000  17326
1  C2000   6074
2  C1200   4837
3  C3000   1918
4  C2100   1883

In [37]:
# Creating the list of classification we would like to replacing, choosing 10 as a cutoff value
class_to_replace = []

for i in class_df.index:
    if class_df.loc[i]['count'] < 10:
        class_to_replace.append(class_df.loc[i]['class'])

print(class_to_replace)

['C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


In [38]:
# Replacing the above in the dataframe
for cls in class_to_replace:
    application5_df['CLASSIFICATION'] = application5_df['CLASSIFICATION'].replace(cls,"Other")
    
# Checking to make sure binning was successful
application5_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [39]:
# Looking at ASK_AMT value counts for binning
ask_amt = application5_df['ASK_AMT'].value_counts()
print(ask_amt)

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [40]:
# Converting the series to a dataframe in order to make a list of the asked amounts we would like to replace
amt_df = ask_amt.to_frame(name='count').reset_index(names='amount')
amt_df.head()

amount  count
0    5000  25398
1   10478      3
2   15583      3
3   63981      3
4    6725      3

In [41]:
# Creating the list of asked amounts we would like to replacing, choosing 10000 as a cutoff value
amt_to_replace = []

for i in amt_df.index:
    if amt_df.loc[i]['count'] < 10000:
        amt_to_replace.append(amt_df.loc[i]['amount'])

print(amt_to_replace)

[10478, 15583, 63981, 6725, 7003, 42072, 5731, 46327, 39122, 11839, 6742, 21264, 86039, 73987, 9537, 29130, 206725, 13420, 136341, 80599, 12584, 7686, 20834, 17614, 31453, 42962, 26432, 18063, 23202, 6960, 5297, 21389, 246851, 94253, 6000, 146824, 12753, 15039, 39168, 47129, 10070, 43814, 34875, 5706, 6279, 17010, 184668, 5470, 54024, 20150, 90834, 120158, 6035, 42178, 13681, 13831, 27073, 64471, 5175, 22579, 83211, 35734, 5170, 36683, 51366, 50890, 34995, 32141, 6920, 8632, 83827, 67367, 59647, 6398, 15722, 31549, 13898, 238933, 7187, 16071, 13499, 24459, 27441, 14000, 18985, 63520, 5179, 14765, 201112, 350000, 32507, 8746, 22472, 7049, 6405, 27944, 34792, 57588, 73627, 32807, 16200, 12238, 64093, 41894, 57957, 34538, 9413, 57689, 15372, 61397, 1427468, 24105, 5130, 5548, 9323, 16435, 19704, 16482, 56135, 37390, 60000, 195661, 18826, 6010, 9255, 7287, 210432, 40429, 17735, 20071, 5255, 21346, 29541, 20537, 5894, 40743, 20684, 25952, 9885, 159223, 80168, 5006, 37489, 30769, 150599, 271

In [42]:
# Converting categorical data to numeric with `pd.get_dummies`
enc_application5_df = pd.get_dummies(application5_df)

In [43]:
enc_application5_df.head()

STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T12  APPLICATION_TYPE_T13  \
0                     1                     0                     0   
1                     0                     0                     0   
2                     0                     0                     0   
3                     0                     0                     0   
4                     0                     0                     0   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T2  APPLICATION_TYPE_T3  ...  \
0                     0                    0                    0  ...   
1                     0                    0                    1  ...   
2                     0                    0                    0  ...   
3                     0                    0                    1  ...   
4                     0                    0                    1  ...   

   INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                  0                       0                         0   
1                  1                       0                         0   
2                  0                       0                         0   
3                  0                       1                         0   
4                  0                       0                         1   

   INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  \
0                   0                 0                       0   
1                   0                 0                       0   
2                   0                 0                       0   
3                   0                 0                       0   
4                   0                 0                       0   

   INCOME_AMT_50M+  INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  \
0                0                  0                         1   
1                0                  0                         1   
2                0                  0                         1   
3                0                  0                         1   
4                0                  0                         1   

   SPECIAL_CONSIDERATIONS_Y  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 72 columns]

In [44]:
# Splitting our preprocessed data into our features and target arrays
y = enc_application5_df['IS_SUCCESSFUL'].values
X = enc_application5_df.drop(['IS_SUCCESSFUL'],1).values 

# Splitting the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 1)

<ipython-input-44-e6a4c46a5c95>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = enc_application5_df.drop(['IS_SUCCESSFUL'],1).values


In [45]:
# Creating a StandardScaler instances
scaler = StandardScaler()

# Fitting the StandardScaler
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [46]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 80)                5760      
                                                                 
 dense_16 (Dense)            (None, 30)                2430      
                                                                 
 dense_17 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,221
Trainable params: 8,221
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [48]:
# Training the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5717 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7314
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7332
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5430 - accuracy: 0.7340
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5424 - accura

In [49]:
# Evaluating the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"5th attempt Loss: {model_loss}, 5th attempt Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5579 - accuracy: 0.7300 - 583ms/epoch - 2ms/step
5th attempt Loss: 0.5579429268836975, 5th attempt Accuracy: 0.7300291657447815


In [51]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")